In [1]:
import tweepy
import json

In [2]:
from itertools import islice

In [4]:
auth = tweepy.auth.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [5]:
print(api)

In [6]:
# The Yahoo! Where On Earth ID for the entire world is 1.
# See https://dev.twitter.com/docs/api/1.1/get/trends/place and
# http://developer.yahoo.com/geo/geoplanet/

WORLD_WOE_ID = 1
ZH_WOE_ID = 784794

world_trends = api.trends_place(WORLD_WOE_ID)
zh_trends = api.trends_place(ZH_WOE_ID)

print(json.dumps(list(islice(world_trends[0]['trends'], 2)), indent=1))

[
 {
  "name": "#PSGMUN",
  "url": "http://twitter.com/search?q=%23PSGMUN",
  "promoted_content": null,
  "query": "%23PSGMUN",
  "tweet_volume": 210851
 },
 {
  "name": "#MUFC",
  "url": "http://twitter.com/search?q=%23MUFC",
  "promoted_content": null,
  "query": "%23MUFC",
  "tweet_volume": 436420
 }
]


In [7]:
world_trends_set = set([trend['name'] 
                        for trend in world_trends[0]['trends']])

zh_trends_set = set([trend['name'] 
                     for trend in zh_trends[0]['trends']])

common_trends = world_trends_set.intersection(zh_trends_set)

print(common_trends)

{'#ChampionsLeague', 'United', 'Neymar', '#PSGMUFC', '#PSGMUN', '#MUFC'}


In [8]:
q = '#ChampionsLeague' 

count = 100

# Import unquote to prevent url encoding errors in next_results
from urllib.parse import unquote

# See https://dev.twitter.com/rest/reference/get/search/tweets

search_results = api.search(q=q, count=count)

In [9]:
statuses = search_results['statuses']

In [10]:
search_results.keys()

dict_keys(['statuses', 'search_metadata'])

In [11]:
# Iterate through 5 more batches of results by following the cursor
for _ in range(5):
    print('Length of statuses', len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=847960489447628799&q=%23RIPSelena&count=100&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    #print(next_results[1:].split("&"))
    search_results = api.search(**kwargs)
    statuses += search_results['statuses']

Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500


In [13]:
# Show one sample search result by slicing the list...
print(json.dumps(statuses[0], indent=4))

{
    "created_at": "Wed Mar 06 22:58:25 +0000 2019",
    "id": 1103429657695784965,
    "id_str": "1103429657695784965",
    "text": "RT @DanielWebber81: And that, ladies and gentlemen... Is what @ManUtd is all about! #OlesAtTheWheel #MUFC #ChampionsLeague",
    "truncated": false,
    "entities": {
        "hashtags": [
            {
                "text": "OlesAtTheWheel",
                "indices": [
                    84,
                    99
                ]
            },
            {
                "text": "MUFC",
                "indices": [
                    100,
                    105
                ]
            },
            {
                "text": "ChampionsLeague",
                "indices": [
                    106,
                    122
                ]
            }
        ],
        "symbols": [],
        "user_mentions": [
            {
                "screen_name": "DanielWebber81",
                "name": "Danny Webber \ud83d\udc1d",
          

In [15]:
for i in range(3):
    print()
    print(statuses[i]['text'])
    print('Favorites: ', statuses[i]['favorite_count'])
    print('Retweets: ', statuses[i]['retweet_count'])


RT @DanielWebber81: And that, ladies and gentlemen... Is what @ManUtd is all about! #OlesAtTheWheel #MUFC #ChampionsLeague
Favorites:  0
Retweets:  1091

Em choque com essa eliminação #ChampionsLeague https://t.co/oWQ2nbOOWQ
Favorites:  0
Retweets:  0

RT @ChrisCowlin: Well, the @ChampionsLeague quarter-final draw will be interesting next week... 😀⚽️👍
- Tottenham Hotspur
- Manchester Utd
-…
Favorites:  0
Retweets:  4
